In [4]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [6]:
%%sql
drop table if exists Materials;
CREATE TABLE Materials
(center         CHAR(12) NOT NULL,
 receive_date   DATE     NOT NULL,
 material       CHAR(12) NOT NULL,
 PRIMARY KEY(center, receive_date));

INSERT INTO Materials VALUES('東京'	,'2018-4-01',	'錫');
INSERT INTO Materials VALUES('東京'	,'2018-4-12',	'亜鉛');
INSERT INTO Materials VALUES('東京'	,'2018-5-17',	'アルミニウム');
INSERT INTO Materials VALUES('東京'	,'2018-5-20',	'亜鉛');
INSERT INTO Materials VALUES('大阪'	,'2018-4-20',	'銅');
INSERT INTO Materials VALUES('大阪'	,'2018-4-22',	'ニッケル');
INSERT INTO Materials VALUES('大阪'	,'2018-4-29',	'鉛');
INSERT INTO Materials VALUES('名古屋',	'2018-3-15',	'チタン');
INSERT INTO Materials VALUES('名古屋',	'2018-4-01',	'炭素鋼');
INSERT INTO Materials VALUES('名古屋',	'2018-4-24',	'炭素鋼');
INSERT INTO Materials VALUES('名古屋',	'2018-5-02',	'マグネシウム');
INSERT INTO Materials VALUES('名古屋',	'2018-5-10',	'チタン');
INSERT INTO Materials VALUES('福岡'	,'2018-5-10',	'亜鉛');
INSERT INTO Materials VALUES('福岡'	,'2018-5-28',	'錫');

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.
1 rows affected.


[]

## やりたいこと
+ 各グループ(拠点)ごとに資材が重複しているような拠点(グループ名)を列挙する

In [7]:
%%sql
-- existsを使って書く
-- existsのほうが出力できる情報量が多い
select distinct center
from Materials as m1
where exists (
    select *
    from Materials as m2
    where m1.center = m2.center and
    m1.receive_date <> m2.receive_date and
    m1.material = m2.material
)

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


center
東京
名古屋


In [8]:
%%sql
-- havingを利用
select center
from Materials
group by center
having count(material) <> count(distinct material);

*  postgresql://padawan:***@db:5432/dsdojo_db
2 rows affected.


center
東京
名古屋
